# C. Increate the number of epochs (5 marks)

Repeat Part B but use 100 epochs this time for training.

How does the mean of the mean squared errors compare to that from Step B?

# Regression model using Keras (Data normalized with epochs 100)

Lets import numpy and pandas to help us load and analyze data

In [1]:
import numpy as np
import pandas as pd

In [2]:
#lets load the data, and take a look at the data using .head()
data = pd.read_csv("concrete_data.csv")
data.head()


,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
#lets print the shape of the data (i.e. number of rows and columns)
data.shape

(1030, 9)

Therefore, our dataset has 1030 rows and only 9 columns.
Lets take a look at the data for any missing values before we start building the model using the data.

In [4]:
data.describe()


,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks good so far, therfore we can begin the next steps.

Since, for the first part, we are not to normalioze the data, I will jump straight to splitting the dataset.

### Lets divide our dataset into predictors (X) and target variable (y) (independent and dependent variable)

In [6]:
X = data[['Cement','Blast Furnace Slag','Fly Ash',
                  'Water','Superplasticizer','Coarse Aggregate','Fine Aggregate','Age']]

X.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [7]:
y = data[['Strength']]
y.head()

,Strength
0,79.99
1,61.89
2,40.27
3,41.05
4,44.30


### Lets convert both, X and y into arrays

In [8]:
X = X.values
X

array([[ 540. ,    0. ,    0. , ..., 1040. ,  676. ,   28. ],
       [ 540. ,    0. ,    0. , ..., 1055. ,  676. ,   28. ],
       [ 332.5,  142.5,    0. , ...,  932. ,  594. ,  270. ],
       ...,
       [ 148.5,  139.4,  108.6, ...,  892.4,  780. ,   28. ],
       [ 159.1,  186.7,    0. , ...,  989.6,  788.9,   28. ],
       [ 260.9,  100.5,   78.3, ...,  864.5,  761.5,   28. ]])

In [9]:
y = y.values
y

array([[79.99],
       [61.89],
       [40.27],
       ...,
       [23.7 ],
       [32.77],
       [32.4 ]])

## Lets Normalize the X (independent variables/predictors)

In [10]:
X_normalized = (X - X.mean()) / X.std() 

In [11]:
print(X_normalized.shape)
print(X.shape)

(1030, 8)
(1030, 8)


### Now that we have both, the target and predictor variabels, lets move onto splitting our dataset.


In [12]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X_normalized, y, test_size=0.3, random_state=42)
print(f"Train Set = {X_train.shape},{y_train.shape}")
print(f"Test Set = {X_test.shape},{y_test.shape}")

Train Set = (721, 8),(721, 1)
Test Set = (309, 8),(309, 1)


30% of the dataset has been reserved for testing as per the instructions

### Lets import some important libraries for building our model

In [13]:
import tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [14]:
#lets define n_cols to be the size of the number of variables in X
n_cols = X_test.shape[1]
print(n_cols)

8


#### Therefore, we will have 8 nodes in the input layer of the ANN.

In [15]:
#lets create our model
def regression_model():
    # create the model
    model = tensorflow.keras.Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile thye model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


The above function creates a model that has one hidden layer with 10 neurons and uses ReLU activation function. It uses the adam optimizer and the mean squared error as the loss function, as per instructions.

In [25]:
#lets build the model
model = regression_model()

Lets train the model with 100 epochs

In [17]:
# fit the model
epochs = 100
model.fit(X_train, y_train, epochs=epochs, verbose=2)

Train on 721 samples
Epoch 1/100
721/721 - 0s - loss: 1543.6724
Epoch 2/100
721/721 - 0s - loss: 1525.0633
Epoch 3/100
721/721 - 0s - loss: 1505.2315
Epoch 4/100
721/721 - 0s - loss: 1483.4702
Epoch 5/100
721/721 - 0s - loss: 1459.7623
Epoch 6/100
721/721 - 0s - loss: 1433.4118
Epoch 7/100
721/721 - 0s - loss: 1404.6180
Epoch 8/100
721/721 - 0s - loss: 1373.1280
Epoch 9/100
721/721 - 0s - loss: 1339.1406
Epoch 10/100
721/721 - 0s - loss: 1302.6001
Epoch 11/100
721/721 - 0s - loss: 1264.0459
Epoch 12/100
721/721 - 0s - loss: 1223.5063
Epoch 13/100
721/721 - 0s - loss: 1181.4640
Epoch 14/100
721/721 - 0s - loss: 1137.7886
Epoch 15/100
721/721 - 0s - loss: 1093.0291
Epoch 16/100
721/721 - 0s - loss: 1047.9777
Epoch 17/100
721/721 - 0s - loss: 1001.6471
Epoch 18/100
721/721 - 0s - loss: 955.8425
Epoch 19/100
721/721 - 0s - loss: 909.9379
Epoch 20/100
721/721 - 0s - loss: 864.5354
Epoch 21/100
721/721 - 0s - loss: 820.0597
Epoch 22/100
721/721 - 0s - loss: 776.2470
Epoch 23/100
721/721 - 0s

In [18]:
#Lets evaluate the model now:

loss_ = model.evaluate(X_test, y_test, verbose  =2)
y_pred = model.predict(X_test)
loss_



309/1 - 0s - loss: 217.8162


246.53869426443353



Now we need to compute the mean squared error between the predicted concrete strength and the actual concrete strength.

Let's import the mean_squared_error function from Scikit-learn.



In [19]:
from sklearn.metrics import mean_squared_error

In [20]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(f"Mean of MSE = {mean}")
print(f"Standard Deviation of MSE = {standard_deviation}")

Mean of MSE = 246.53869985959182
Standard Deviation of MSE = 0.0


### Now, we will repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors and calculate the mean and Standard deviation of the list.

In [21]:
z =1 #for indexing 
mse_list_50 = [] #empty list for the 50 values 
model = regression_model()
epochs = 100
for x in range(50):
    
    X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.3, random_state=x)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    loss_1 = model.evaluate(X_test, y_test, verbose  =0)
    print(f" {z}: MSE = {loss_1}")
    y_pred1 = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred1)
    mse_list_50.append(mean_square_error)
    z += 1
    
#lets convert the list mse_list_50 into array before we calculate the mean and the standard deviation of the mean squared errors.
mse_array_50 = np.array(mse_list_50)
mse_array_50_mean = np.mean(mse_array_50)
mse_array_50_std = np.std(mse_array_50)
print(f"Mean of all 50 Mean squared error values = {mse_array_50_mean}")
print(f"Standard Deviation of all 50 Mean squared error values = {mse_array_50_std}")

 1: MSE = 236.84799564694896
 2: MSE = 203.99897301930054
 3: MSE = 181.88434099610956
 4: MSE = 160.05496457788166
 5: MSE = 132.1591489230159
 6: MSE = 106.0733888990285
 7: MSE = 118.07195136307898
 8: MSE = 88.93317163955047
 9: MSE = 101.44817502907564
 10: MSE = 90.62083760974477
 11: MSE = 82.0717632701096
 12: MSE = 72.30988803110462
 13: MSE = 81.49140453647256
 14: MSE = 80.86082895519664
 15: MSE = 72.2196870081633
 16: MSE = 59.94403839111328
 17: MSE = 66.58282897850457
 18: MSE = 59.575076353202746
 19: MSE = 53.23358559222669
 20: MSE = 64.38335769307652
 21: MSE = 53.90175147195464
 22: MSE = 53.3521104954593
 23: MSE = 51.796442507154346
 24: MSE = 49.195285142818314
 25: MSE = 50.82791908041945
 26: MSE = 49.758763557113106
 27: MSE = 50.6227720069268
 28: MSE = 49.535190866217256
 29: MSE = 54.11487567000405
 30: MSE = 48.872108570580345
 31: MSE = 46.58675730189845
 32: MSE = 44.68258185525542
 33: MSE = 44.858199134999495
 34: MSE = 48.38930071981979
 35: MSE = 48.

In [22]:
#a look at the list of 50 mean squared errors
mse_list_50

[236.84799462734708,
 203.9989726439307,
 181.8843332357169,
 160.05495830980843,
 132.15914691125798,
 106.07339026307939,
 118.07194858819075,
 88.93317288673165,
 101.44817528694978,
 90.6208364649111,
 82.07176238243215,
 72.30988675026855,
 81.49140527437089,
 80.86082887640795,
 72.2196871251297,
 59.94403722259976,
 66.58282703297684,
 59.57507283702592,
 53.23358665727842,
 64.38335691569061,
 53.901750069920865,
 53.35210929509781,
 51.796441299697456,
 49.19528705814005,
 50.82791717004377,
 49.75876431167775,
 50.622771758489684,
 49.53518768608867,
 54.11487616941146,
 48.872107027132536,
 46.586758747327,
 44.68258180616781,
 44.85819924151685,
 48.389301050091355,
 48.20616259531685,
 53.74377320424668,
 50.8398340628905,
 52.40538248400962,
 46.19766637871455,
 43.13790491985813,
 49.087282267805314,
 43.194975302007755,
 45.808549059001656,
 51.53484612355477,
 50.75570699285552,
 50.99836415122469,
 50.85958848424584,
 47.883798858434254,
 51.79501603711131,
 49.466592

## How does the mean of the mean squared errors compare to that from Step B?

In [28]:
print("With epochs 50- Part B")
print(f"\nMean of all 50 Mean squared error values = 105.14580754892266")
print(f"Standard Deviation of all 50 Mean squared error values = 45.379225803390646")

With epochs 50- Part B

Mean of all 50 Mean squared error values = 105.14580754892266
Standard Deviation of all 50 Mean squared error values = 45.379225803390646


In [30]:
print("With epochs 100- Part C")
print(f"\nMean of all 50 Mean squared error values = {mse_array_50_mean}")
print(f"Standard Deviation of all 50 Mean squared error values = {mse_array_50_std}")

With epochs 100- Part C

Mean of all 50 Mean squared error values = 71.90349752100936
Standard Deviation of all 50 Mean squared error values = 42.28821962650301
